In [1]:
from Nexullance_OPT import Nexullance_OPT
from Nexullance_MP import Nexullance_MP
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from topologies.DDF import DDFtopo
import globals as gl
import numpy as np

In [2]:
config=(36, 5)
EPR=3
ddf_network = DDFtopo(config[0], config[1])

In [3]:
Cap_remote = 10 #GBps
Cap_local = 10 #GBps
M_EPs = gl.generate_half_shift_traffic_pattern(config[0], EPR) # a matrix filled with 1s
scaling_factor = 3
M_EPs = scaling_factor * M_EPs
M_R = gl.convert_M_EPs_to_M_R(M_EPs, config[0], EPR)

In [4]:
ASP = ddf_network.calculate_all_shortest_paths()
ECMP_ASP = gl.ECMP(ASP)
remote_link_flows, local_link_flows = ddf_network.distribute_M_EPs_on_weighted_paths(ECMP_ASP, EPR, M_EPs)
max_remote_link_load = np.max(remote_link_flows)/Cap_remote
max_local_link_load = np.max(local_link_flows)/Cap_local
print("Max remote link load: ", max_remote_link_load)
print("Max local link load: ", max_local_link_load)
if max_remote_link_load > 1.0 or max_local_link_load > 1.0:
    print("network congestion may present")
    mu = max_local_link_load/max_remote_link_load
    if mu > 1.0:
        print(f"mu = {mu} > 1, max local link load is higher")
    else:
        print(f"mu = {mu} < 1, max remote link load is higher")
else:
    print("network is probably not congested")
print("network_total_throughput = ", gl.network_total_throughput(M_EPs, max_remote_link_load, max_local_link_load), "GBps")


Max remote link load:  1.8
Max local link load:  0.3
network congestion may present
mu = 0.16666666666666666 < 1, max remote link load is higher
network_total_throughput =  180.0 GBps


In [5]:
nexu_opt = Nexullance_OPT(ddf_network.nx_graph, M_R, Cap_remote, 0, False)
nexu_opt.init_model()
new_max_remote_link_load = nexu_opt.solve()
print("updated max remote link load: ", new_max_remote_link_load)
print("network_total_throughput = ", gl.network_total_throughput(M_EPs, new_max_remote_link_load, max_local_link_load), "GBps")


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2411299
Academic license 2411299 - for non-commercial use only - registered to zi___@ugent.be
Set parameter Threads to value 1
Optimal solution found
updated max remote link load:  0.7200000000000003
network_total_throughput =  324.0 GBps


In [6]:
nexu_opt = Nexullance_MP(ddf_network.nx_graph, ASP, M_R, Cap_remote, 0, False)
nexu_opt.init_model()
new_max_remote_link_load, weighted_path_dict = nexu_opt.solve()
print("updated max remote link load: ", new_max_remote_link_load)
print("network_total_throughput = ", gl.network_total_throughput(M_EPs, new_max_remote_link_load, max_local_link_load), "GBps")


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2411299
Academic license 2411299 - for non-commercial use only - registered to zi___@ugent.be
Set parameter Threads to value 1


Optimal solution found
updated max remote link load:  1.8
network_total_throughput =  180.0 GBps


In [7]:
APST_4 = ddf_network.calculate_all_paths_within_length(4)
nexu_opt = Nexullance_MP(ddf_network.nx_graph, APST_4, M_R, Cap_remote, 0, False)
nexu_opt.init_model()
new_max_remote_link_load, weighted_path_dict = nexu_opt.solve()
print("updated max remote link load: ", new_max_remote_link_load)
print("network_total_throughput = ", gl.network_total_throughput(M_EPs, new_max_remote_link_load, max_local_link_load), "GBps")


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2411299
Academic license 2411299 - for non-commercial use only - registered to zi___@ugent.be
Set parameter Threads to value 1
Optimal solution found
updated max remote link load:  0.7499999999999997
network_total_throughput =  324.0 GBps
